Preprocessing data

In [ ]:
import csv
import numpy

dataFileName = "AmesHousingSubset.csv"
def load_data():
  '''loads the data from the data file into a 2d 
  list and the feature names into a list and returns both
  Also replaces empty values with -1'''
  # featureList = []
  data = []
  with open(dataFileName,"r") as fh:
    reader = csv.reader(fh)
    first = True
    x_train = []
    x_val = []
    y_train = []
    y_val = []
    x_test = []
    y_test = []
    y = []
    for row in reader:
      splitRow = row[2:25]
      splitRow.append(row[27]) #6:len(row)-2]
      if first:
        first = False
        # featureList.extend(splitRow)
        continue
      y.append(row[1])
      # Go through row and replace empty values with -1
      for i, ele in enumerate(splitRow):
        if ele == "":
          splitRow[i] = "-1"
      data.append(splitRow)
    # print("y:", y)
    # split into training and validation
    for i, instance in enumerate(data):
      if len(x_train) < 800:
        x_train.append(instance)
        y_train.append(y[i])
        continue
      if len(x_val) < 300:
        x_val.append(instance)
        y_val.append(y[i])
        continue
      x_test.append(instance)
      y_test.append(y[i])
  print(len(x_test))
  return x_train, x_val, y_train, y_val,x_test,y_test

def convert_strings_to_floats(data):
  '''converts the string values into '''
  conversionTable = {
  "Po": 0.0, "Fa" : 0.25, "TA" : 0.5, "Gd" : 0.75, "Ex" : 1.0, 
  "FV" : 0.0, "RL" : 0.25, "RM" : 0.5, "RH" : 0.75, "C (all)" : 1.0, 
  "Slab" : 0.0, "PConc" : 0.2, "CBlock" : 0.4, "Stone" : 0.6, "Wood" : 0.8, "BrkTil" : 1.0, 
  "Sev" : 0.0, "Maj2" : 0.167, "Maj1" : 0.334, "Mod" : 0.501, "Min2" : 0.668, "Min1" : 0.835, "Typ" : 1.0, 
  "Y" : 1.0, "P" : 0.5, "N" : 0.0
  }
  for row in range(len(data)):
    for col in range(len(data[row])):
      if data[row][col] in conversionTable:
        # print("Before:",data[row][col])
        data[row][col] = conversionTable.get(data[row][col])
        # print("After:", data[row][col])
  return data

def replace_missing_vals(data):
  '''replaces all of the -1 values which were once 
  missing with the mean of the column'''
  #print("hi")
  #print(len(data))
  for column in range(len(data[0])):
    rollingSum = 0.0
    numVals = 0
    for row in range(len(data)):
      num = data[row][column]
      if not num == -1:
        rollingSum += num
        numVals += 1
    mean = rollingSum/numVals
    for row in range(len(data)):
      #print("Before:", row)
      if(data[row][column] == -1):
        data[row][column] = mean
      #print("After:", data[row])
  return data

def scale(data):
  '''takes all of the numerical values and 
  scales them down to a value between 0 and 1'''
  for col in range(len(data[1])):
    max = 0
    for row in range(len(data)):
      if(data[row][col]>max):
        max = data[row][col]
    if max==0:
      continue
    for row in range(len(data)):
      data[row][col] /= max
  return data

def getCleanData():
  # code in here will be run by main.py
  x_train, x_val, y_train, y_val, x_test, y_test= load_data()
  # print("Features:", feature)
  print("Testing replacing string with val:")
  x_train = convert_strings_to_floats(x_train)
  x_val = convert_strings_to_floats(x_val)
  x_test = convert_strings_to_floats(x_test)
  print("Testing replacing:")
  # print(type(data))
  # for row in data:
  #   print(len(row))
  #   if len(row) == 28:
  #     print(row)
  x_train = replace_missing_vals([list(map(int, row)) for row in x_train])
  x_val = replace_missing_vals([list(map(int, row)) for row in x_val])
  x_test = replace_missing_vals([list(map(int, row)) for row in x_test])

  print("Testing scaling:")
  x_train = scale(x_train)
  x_val = scale(x_val)
  x_test = scale(x_test)
  y_train = list(map(int, y_train))
  y_val = list(map(int, y_val))
  y_test = list(map(int, y_test))
  # print("x_train:", numpy.array(x_train))
  # print("x_val:", numpy.array(x_val))
  # print("y_train:", numpy.array(y_train))
  # print("y_val:", numpy.array(y_val))

  print("Data is ready for linear regression!!!")
  return x_train, x_val, x_test, y_train, y_val, y_test

Code for linear regression

In [ ]:
import numpy

cost_history = []
accuracy_history = []
cost_history_val = []
accuracy_history_val = []

def r2(pred,y):
  sum_y = 0
  for i in y:
    sum_y+=i
  print(sum_y)
  ssto = 0
  mean_y = sum_y/len(y)
  for i in y:
    ssto+=(mean_y-i)**2
  sse = 0
  for i in range(len(pred)):
    sse += (pred[i]-y[i])**2
  return 1-sse/ssto
                     

def getPrediction(x, w):
  '''h(x) is the prediction based on inputs and weights'''
  return float(numpy.sum(numpy.dot(x,w)))

def costRow(x,w,y,p):
  '''difference between predicted and actual value
  '''
  pred = getPrediction(x,w)
  dif = (pred - y)
  # if p:
    # print("prediction: "+str(pred)+" actual: "+str(y) + " dif: " + str(dif))
  return dif

def costDeriv(x, w, y, learning_rate, feature):
  '''finds the derivative for gradient descent
  '''
  sum = 0
  for row in range(len(x)):
    sum += costRow(x[row], w, y[row],False) * x[row][feature]
  # sum *= learning_rate
  sum /= len(x)
  return sum

def costMat(x,w,y,p):
  '''sum squared error
  '''
  total = 0
  totalPercent = 0
  for row in range(len(x)):
    #total += (costRow(x[row],w,y[row],p)**2)
    num = abs(costRow(x[row],w,y[row],p))
    total+=num
    totalPercent+=num/y[row]
  average_error = total/len(x)
  # average_cost = numpy.sum(y)/len(y)
  accuracy = 100-totalPercent/len(x)*100
  return average_error, accuracy # ((average_cost - average_error)/average_cost) * 100

def regression(x,y,x_val,y_val,learning_rate=2):
  '''Runs multi-variate linear regression algorithm
  '''
  w = [0 for i in range(len(x[0]))]
  iterationNum = 0
  while iterationNum < 1250: # 400 times per instance
    for feature in range(len(w)): # iterate through each feature in row
      jDeriv = costDeriv(x,w,y,learning_rate,feature)
      # print(jDeriv)
      w[feature] = w[feature] - learning_rate * jDeriv
    cost, costPercent = costMat(x,w,y,iterationNum%50==0)
    
    cost_history.append(cost)
    accuracy_history.append(costPercent)
    
    cost_val, cost_percent_val = costMat(x_val,w,y_val,iterationNum%50==0)
    cost_history_val.append(cost_val)
    accuracy_history_val.append(cost_percent_val)
    
    if iterationNum % 10 == 0:
      print("Iteration #", iterationNum, ":", cost, " percent:", costPercent)
      # print("bias:", w[0])
    iterationNum += 1
  return w

Training and Testing

In [ ]:
import matplotlib.pyplot as plt

def with_bias(x):
  '''Add column of 1s to x for bias'''
  for i in range(len(x)):
    with_bias = [1.0]
    with_bias.extend(x[i])
    x[i] = with_bias
  return x

def plot_error(cost_history):
  print("cost history:", cost_history)
  plt.plot(cost_history, color = "red")
  plt.ylabel('Mean error (dollars)')
  plt.xlabel('Iterations')
  plt.savefig("cost-bias.jpg")
  plt.show()

def plot_accuracy(accuracy_history):
  plt.plot(accuracy_history, color = "blue")
  plt.ylabel('Relative Accuracy (percent)')
  plt.xlabel('Iterations')
  plt.savefig("accuracy-bias.jpg")
  plt.show()

In [ ]:
def train(x, y,x_val,y_val):
  print("hi")
  w = regression(x,y,x_val,y_val)
  print("done!")
  plot_error(cost_history)
  plot_accuracy(accuracy_history)
  plot_error(cost_history_val)
  plot_accuracy(accuracy_history_val)
  print("training r2: ", r2(numpy.dot(x,w),y))
  return w
 
def test(x,y,w):
  #x= with_bias(x)
  print("average validation:", costMat(x, w, y, False))
#   print("average validation:", r2(numpy.dot(x,w),y))
  

In [ ]:
def run_train_and_test():
  cost_history.clear()
  accuracy_history.clear()
  x_train, x_val, x_test, y_train, y_val, y_test = getCleanData()
  x_train = with_bias(x_train)
  x_test = with_bias(x_test)
  x_val = with_bias(x_val)
  w = train(x_train, y_train,x_val,y_val)
  error, accuracy = costMat(x_test,w,y_test,False)
  print("Test error: ", error)
  print("Test accuracy: ", accuracy)
  #test(x_val, y_val, w)

In [ ]:
run_train_and_test()

360
Testing replacing string with val:
Testing replacing:
Testing scaling:
Data is ready for linear regression!!!
hi
Iteration # 0 : 64001.709794613096  percent: 57.776973375447575
Iteration # 10 : 36415.24260581442  percent: 76.6407985279007
Iteration # 20 : 33878.18882454305  percent: 78.40690074240479
Iteration # 30 : 33219.03720443806  percent: 78.84326391922365
Iteration # 40 : 32837.67980461558  percent: 79.11270829643952
Iteration # 50 : 32528.680495370667  percent: 79.31189531006348
Iteration # 60 : 32247.980587282153  percent: 79.4807730348138
Iteration # 70 : 31985.76495109652  percent: 79.63980644729124
Iteration # 80 : 31744.14323182462  percent: 79.78602439489283
Iteration # 90 : 31520.531305341803  percent: 79.91959366125843
Iteration # 100 : 31318.95955645072  percent: 80.03731324300607
Iteration # 110 : 31125.150286130644  percent: 80.14979868346744
Iteration # 120 : 30936.871397651586  percent: 80.25913057484145
Iteration # 130 : 30753.85302104468  percent: 80.36642797

KeyboardInterrupt: ignored